# Introdução e o Problema

| **Fase**                  | **Status**                                                          | **Descrição**                                                                               |
| ------------------------- | ------------------------------------------------------------------- | ------------------------------------------------------------------------------------------- |
| **Pré-Contrato**          | Approved (Aprovado)                                                 | Aprovado pelo Banco Mundial, ainda não assinado.                                            |
| **Assinatura**            | Signed (Assinado)                                                   | Contrato assinado, mas pendente de efetivação (condições precedentes ainda não cumpridas).  |
| **Início**                | Effective (Vigente / Efetivado)                                     | Contrato em vigor, habilitado para iniciar desembolsos.                                     |
| **Execução**              | Disbursing (Em Processo de Desembolso)                              | Valores sendo gradualmente liberados conforme o projeto avança.                             |
| **Execução/Amortização**  | Disbursing & Repaying (Simultaneamente Desembolsando e Amortizando) | Parte dos recursos já foi liberada e já começou o pagamento de parcelas desses desembolsos. |
| **Pós-Desembolso**        | Fully Disbursed (Totalmente Desembolsado)                           | Todo o valor contratado foi desembolsado; seguem pagamentos.                                |
| **Amortização**           | Repaying (Em Amortização)                                           | Apenas pagamentos em andamento; não há mais desembolsos.                                    |
| **Quitado**          | Fully Repaid (Totalmente Quitado)                                   | Toda a dívida quitada; contrato encerrado.                                                  |
| **Encerramento especial** | Fully Cancelled (Totalmente Cancelado)                              | Contrato cancelado (total ou parcialmente) antes de execução completa.                      |
| **Encerramento especial** | Fully Transferred (Totalmente Transferido)                          | Transferido para outra entidade ou instrumento financeiro.                                  |
| **Encerramento especial** | Terminated (Encerrado)                                              | Encerrado por rescisão ou fim do prazo contratual.                                          |


| Código   | Significado (Inglês)       | Tradução/Descrição no contexto do IBRD                                                                                                                                                                            |
| -------- | -------------------------- | ----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **FSL**  | Fixed Spread Loan          | **Empréstimo com Spread Fixo**<br>Empréstimo onde o spread (margem sobre a taxa base, ex: LIBOR, SOFR) permanece fixo durante toda a vigência do contrato.                                                        |
| **CPL**  | Currency Pool Loan         | **Empréstimo em Pool de Moedas**<br>Modelo antigo, em que o empréstimo era indexado a um pool de moedas (dólar, euro, iene, etc.). Não são mais contratados atualmente, mas muitos ainda estão sendo amortizados. |
| **NPL**  | New Pool Loan              | **Novo Empréstimo em Pool**<br>Modelo posterior ao CPL, também baseado em cesta de moedas, mas com estrutura de risco e prazos atualizados.                                                                       |
| **SCL**  | Single Currency Loan       | **Empréstimo em Moeda Única**<br>Empréstimos denominados em uma única moeda (por ex.: apenas USD ou EUR), oferecendo mais previsibilidade cambial ao mutuário.                                                    |
| **SCPD** | Special Currency Product D | **Produto Especial de Moeda D**<br>Produto especial, utilizado em algumas operações diferenciadas (por ex.: garantias, financiamentos híbridos).                                                                  |
| **SCPM** | Special Currency Product M | **Produto Especial de Moeda M**<br>Semelhante ao anterior, com diferenças contratuais no tipo de hedge, risco e objetivo.                                                                                         |
| **GURB** | Guarantee – Bank           | **Garantia – Banco**<br>Operação em que o Banco Mundial não empresta recursos diretamente, mas fornece garantia soberana ao mutuário perante outros financiadores.                                                |
| **BLNR** | Blend Loan Non-Rebatable   | **Empréstimo Blend Não-Reembolsável**<br>Modelo específico que mistura recursos de diferentes fontes (por ex.: concessional e não-concessional), sem reembolso de parte dos subsídios.                            |
| **SCPY** | Special Currency Product Y | **Produto Especial de Moeda Y**<br>Outro tipo de produto híbrido, com regras específicas de risco, câmbio ou garantias.                                                                                           |
| **GUBF** | Guarantee – Bank Funded    | **Garantia com Recursos de Fundo**<br>Garantias com fundos vinculados, como mecanismos fiduciários do Banco Mundial.                                                                                              |
| **BLNC** | Blend Loan Concessional    | **Empréstimo Blend Concessional**<br>Combinação de financiamento com termos mais favoráveis (subsídios, taxas reduzidas), geralmente usado para países de renda média-baixa.                                      |


## Classificação entre Estados Terminais

1. Classificar os registros de projetos em estados temporários (não terminais) em algum dos seguintes estados terminais:
    - Totalmente Quitado
    - Totalmente Cancelado
    - Totalmente Transferido
    - Encerrado
    - Totalmente Desembolsado

2. Features que podem enviesar o modelo:
    - `cancelled_amount`
    - `disbursed_amount`
    - `repaid_to_ibrd`
    - `last_disbursement_date` e estados de loan_status `Approved`, `Signed` e `Effective`.

## Objetivo

A partir do dataset antigo, desejamos prever o status de um empréstimo temporário tendo como referência o status final do dataset mais novo.

# Importand os Dados

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
# Carregando dados, renomeando as colunas e filtrando registros que não entrarão na análise
df = pd.read_csv('dataset_final.csv')
df.columns = df.columns.str.replace('/', ' ').str.replace(' ', '_').str.lower().str.removesuffix('_(us$)').str.removesuffix('_(most_recent)').str.replace("__economy_", "").str.replace("__", "_").str.replace("__", "_")
# final_status = ['Terminated', 'Fully Repaid', 'Fully Cancelled', 'Fully Transferred', 'Fully Disbursed']
# df = df[~df['loan_status'].isin(final_status)]
# df = df[df['last_loan_status'].isin(final_status)]

df['last_loan_status'] = df['last_loan_status'].map(lambda x: 1 if x == 'Fully Repaid' else 0)

df = df.reset_index(drop=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6945 entries, 0 to 6944
Data columns (total 34 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   end_of_period              6945 non-null   object 
 1   loan_number                6945 non-null   object 
 2   region                     6945 non-null   object 
 3   country_code               6944 non-null   object 
 4   country_economy            6945 non-null   object 
 5   borrower                   6904 non-null   object 
 6   guarantor_country_code     6732 non-null   object 
 7   guarantor                  6718 non-null   object 
 8   loan_type                  6945 non-null   object 
 9   loan_status                6945 non-null   object 
 10  interest_rate              6809 non-null   float64
 11  currency_of_commitment     0 non-null      float64
 12  project_id                 6924 non-null   object 
 13  project_name               6765 non-null   objec

In [3]:
df_9 = _dntk.execute_sql(
  'select last_loan_status, count(*) from df group by last_loan_status',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_9

,last_loan_status,count_star()
0,0,2749
1,1,4196


# Análise de Atributos Faltantes

In [4]:
# Atributos com dados faltantes
df[df.columns[df.isnull().any()]].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6945 entries, 0 to 6944
Data columns (total 15 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   country_code            6944 non-null   object 
 1   borrower                6904 non-null   object 
 2   guarantor_country_code  6732 non-null   object 
 3   guarantor               6718 non-null   object 
 4   interest_rate           6809 non-null   float64
 5   currency_of_commitment  0 non-null      float64
 6   project_id              6924 non-null   object 
 7   project_name            6765 non-null   object 
 8   first_repayment_date    6893 non-null   object 
 9   last_repayment_date     6893 non-null   object 
 10  agreement_signing_date  5396 non-null   object 
 11  board_approval_date     6944 non-null   object 
 12  effective_date          5526 non-null   object 
 13  closed_date             6937 non-null   object 
 14  last_disbursement_date  3064 non-null   

In [5]:
df_4 = _dntk.execute_sql(
  'select region, count(*) from df group by region order by count(*) desc',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_4

,region,count_star()
0,LATIN AMERICA AND CARIBBEAN,2145
1,EUROPE AND CENTRAL ASIA,1461
2,EAST ASIA AND PACIFIC,1457
3,MIDDLE EAST AND NORTH AFRICA,837
4,AFRICA,537
5,SOUTH ASIA,448
6,EASTERN AND SOUTHERN AFRICA,29
7,AFRICA EAST,14
8,WESTERN AND CENTRAL AFRICA,13
9,AFRICA WEST,4


- country e country_code parecem são a mesma feature, mas a última tem 3 registros faltantes. Talvez de para preencher manualmente.

- Borrower parece ser uma features importante, mas tem uma quantidade razoável de dados faltantes.

- guarantor e guarantor_country_code possuem uma quantodade razoavel de dados faltantes

- interest_rate pode ser preenchida com uma medida de centralidade 

- Eliminar a feature currency_of_cimmitment por todos os valores serem nulos

- Tanto project_name quanto priject_id possuem dados faltantes, mas para questões de análise e de treinamento não são relevante, pois servem apenas à identificação

- Todas as features de data tem dados faltantes que podem ser eventualmente preenchidos. A exceção élast_disbursement_data, que tem muitos dados faltantes e talvez seja eliminada.

# Analisando Variância

In [6]:
# Analisando features numéricas
df.describe(include=[np.number])

,interest_rate,currency_of_commitment,original_principal_amount,cancelled_amount,undisbursed_amount,disbursed_amount,repaid_to_ibrd,due_to_ibrd,exchange_adjustment,borrower's_obligation,sold_3rd_party,repaid_3rd_party,due_3rd_party,loans_held,last_loan_status
count,6809.000000,0.0,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6.945000e+03,6945.0,6.945000e+03,6945.000000
mean,3.593569,NaN,1.110869e+08,7.748708e+06,5.832798e+07,4.530335e+07,2.648377e+07,1.865986e+07,5.457976e+05,1.920566e+07,3.248712e+05,3.248712e+05,0.0,7.698784e+07,0.604176
std,3.496494,NaN,1.875817e+08,4.411216e+07,1.531248e+08,1.183511e+08,7.221890e+07,9.424453e+07,9.362200e+06,9.704344e+07,2.307838e+06,2.307838e+06,0.0,1.780229e+08,0.489062
min,0.000000,NaN,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-1.633040e+07,-2.269229e+07,-2.000000e-02,0.000000e+00,0.000000e+00,0.0,-1.633040e+07,0.000000
25%,0.000000,NaN,1.460000e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000
50%,3.800000,NaN,4.214342e+07,0.000000e+00,0.000000e+00,9.281563e+06,4.085377e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000e+00,1.000000
75%,7.000000,NaN,1.250000e+08,9.353115e+05,3.227100e+07,3.924633e+07,2.465700e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,7.499246e+07,1.000000
max,11.600000,NaN,3.750000e+09,1.995000e+09,3.379011e+09,3.000000e+09,3.000000e+09,2.000000e+09,5.065767e+08,2.000000e+09,6.440199e+07,6.440199e+07,0.0,3.750000e+09,1.000000


- _Due 3rd Party (US)_ possui todos os valores iguais a 0. Será descartada

- _Udisbursed Amount (US)_ possui mais de 75% de valores iguais a 0, porém o desvio-padrão é alto. Investigar mais a sua importância

- Com exceção de _Disbursed Amount (US)_ e _Repaid to IRBD (US)_, as demais features numéricas tem muitos valores iguais a zero. Investigar mais também

In [7]:
# Analisando features não numéricas
df.describe(include=['object'])

,end_of_period,loan_number,region,country_code,country_economy,borrower,guarantor_country_code,guarantor,loan_type,loan_status,project_id,project_name,first_repayment_date,last_repayment_date,agreement_signing_date,board_approval_date,effective_date,closed_date,last_disbursement_date
count,6945,6945,6945,6944,6945,6904,6732,6718,6945,6945,6924,6765,6893,6893,5396,6944,5526,6937,3064
unique,162,6945,10,141,148,872,124,127,11,9,5635,4879,1552,1634,3176,3162,3734,1067,2087
top,2011-04-30,IBRD72520,LATIN AMERICA AND CARIBBEAN,ID,Indonesia,Ministry of Finance,ID,Indonesia,FSL,Repaid,P178946,EDUCATION II,1901-01-03,2001-01-15,2009-07-06,2009-07-06,2009-07-06,1998-06-30,07/06/2009
freq,5192,1,2145,463,463,1733,463,463,2425,3519,9,25,93,28,17,17,17,83,22


- Considerando-se que há 193 países membros na ONU, 142 países distintos parece razoável de se considerar.

- loan_number serve como chave primária dos registros, mas não é relevante para análise ou treinamento. O mesmo vale para project_number e project_id

- As features de data possuem variância razoável e acho que todas podem ser mantidas para a maioria dos problemas, pois indicam movimentações dos empréstimos, sendo importantes para prever por exemplo: data do primeiro/último pagamento, data da próxima movimentação, se o empréstimo já foi pago ou não (empréstimo recentes provavelmente não foram pagos ainda)

## Descartando Dados Numéricos com baixa Variabilidade

In [8]:
df_3 = _dntk.execute_sql(
  'SELECT * from  df;',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_3

,end_of_period,loan_number,region,country_code,country_economy,borrower,guarantor_country_code,guarantor,loan_type,loan_status,...,due_3rd_party,loans_held,first_repayment_date,last_repayment_date,agreement_signing_date,board_approval_date,effective_date,closed_date,last_disbursement_date,last_loan_status
0,2017-02-28,IBRDG2460,LATIN AMERICA AND CARIBBEAN,AR,Argentina,None,None,None,GURB,Approved,...,0,480000000,None,None,None,2017-02-28,None,2020-08-01,None,0
1,2019-05-31,IBRDG2870,EUROPE AND CENTRAL ASIA,GE,Georgia,MINISTRY OF FINANCE - Georgia,GE,Georgia,GURB,Approved,...,0,48902843,None,None,None,2019-05-23,None,2024-12-31,None,0
2,2024-02-29,IBRDG4160,EUROPE AND CENTRAL ASIA,TR,Turkiye,TURKIYE IHRACAT KREDI BANKASI A.S.,TR,Turkiye,GUBF,Approved,...,0,650040000,None,None,None,2024-02-22,2024-09-30,2029-06-30,None,0
3,2011-04-30,IBRD31750,SOUTH ASIA,IN,India,CONTROLLER OF AID ACCOUNTS & AUDIT,IN,India,POOL LOAN,Cancelled,...,0,0,1995-11-01,2010-05-01,1991-01-11,1990-03-06,1991-05-10,1997-06-30,None,0
4,2011-04-30,IBRD74270,EAST ASIA AND PACIFIC,ID,Indonesia,Ministry of Finance,ID,Indonesia,FSL,Disbursing,...,0,32800000,1983-04-09,2026-09-15,2007-03-28,2007-02-13,2007-06-28,2012-06-30,12/02/2010,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6940,2011-04-30,IBRD36000,EUROPE AND CENTRAL ASIA,PL,Poland,MINISTERSTWO ROLNICTWA,PL,Poland,POOL LOAN,Repaid,...,0,0,1997-11-15,2010-05-15,1993-09-08,1993-05-04,1993-09-29,1996-03-31,08/12/1996,1
6941,2011-04-30,IBRD37480,EAST ASIA AND PACIFIC,CN,China,Ministry of Finance,CN,China,POOL LOAN,Repaid,...,0,0,2000-01-15,2014-07-15,1994-07-20,1994-06-07,1994-10-24,2000-06-30,01/25/2001,1
6942,2011-04-30,IBRD38836,LATIN AMERICA AND CARIBBEAN,CL,Chile,Ministerio de Hacienda,CL,Chile,SNGL CRNCY,Repaid,...,0,0,2001-01-15,2010-07-15,1995-10-31,1995-05-09,1996-03-26,2001-06-30,05/17/2001,1
6943,2011-04-30,IBRD47660,EUROPE AND CENTRAL ASIA,RU,Russian Federation,Ministry of Finance,RU,Russian Federation,SNGL CRNCY,Disbursed,...,0,119790000,1997-04-04,2022-05-15,2005-06-03,2005-02-24,2005-07-01,2006-12-31,12/27/2006,1


In [9]:
df_mod = df.drop(columns=[
        'interest_rate',
        'currency_of_commitment',
        'original_principal_amount',
        'cancelled_amount',
        'due_3rd_party',
        'disbursed_amount',
        'repaid_to_ibrd',
        ])

def little_variance(df, limit):
    # Para cada coluna, verifica se o valor mais frequente representa mais do que limit% dos dados
    # e retorna uma lista com os alias das colunas que atendem ao critério
    cols_little_variance = [
        col for col in df.columns 
        if df[col].value_counts(normalize=True).iloc[0] > limit
    ]
    return cols_little_variance

little_variance(df_mod, 0.70)

['end_of_period',
 'due_to_ibrd',
 'exchange_adjustment',
 "borrower's_obligation",
 'sold_3rd_party',
 'repaid_3rd_party']

Todos os atributos em questão apresentam mais de 70% dos registros com o mesmo valor
configurando uma variabilidade baixíssima.

## Analisando Relação entre Tomadores e Garantidores

In [10]:
df_mod.query('country_code != guarantor_country_code')[['country_economy', 'guarantor']]

,country_economy,guarantor
0,Argentina,NaN
41,China,China
76,"Yugoslavia, former",NaN
166,Portugal,NaN
168,"Yugoslavia, former",NaN
...,...,...
6865,Austria,NaN
6885,Singapore,NaN
6901,"Congo, Democratic Republic of",Belgium
6917,"Yugoslavia, former",NaN


In [11]:
df_mod.query('country_code != guarantor_country_code and guarantor_country_code == guarantor_country_code and guarantor == guarantor').shape[0]

27

Dos 641 empréstimos em que o país do tomador difere do país do garantidor, 361 registros possuem valores não nulos nos campos de nome e código do país garantidor. Considerando que, na maior parte dos casos, o garantidor é um país e coincide com o país do tomador, os campos referentes ao garantidor (nome e código) aparentam ter baixa relevância analítica. Assim, sua remoção do conjunto de dados pode ser realizada sem perdas significativas de informação.

## Determinando Features a serem Descartadas

Com essa análise preliminar, descartaremos as seguintes features por baixa variabilidade 

- _currency_of_commitment_
- _due_3rd_party_
- _undisbursed_amount_
- _sold_3rd_party_
- _repaid_3rd_arty_
- _end_of_period_
- _due_to_IRBD _
- _exchage_djustment _
- _borrower's_obligation _
- _loans_held_

A Features que estão diretamente ou indiretamente relacionada a outra
- _guarantor_country e guarantor_country_ variam muito pouco em relação a _country_ode_
- _country_ pode ser diretamente resgatado de _country_code_
- _project_name_ pode ser resgatada depois com _project_id_

Features de identificação, mas sem relevância anlítica
- _loan_number_
- _project_id_

https://www.kaggle.com/datasets/tunguz/ibrd-statement-of-loans-historical-data

# Análise de Correlação

In [12]:
df_filtered = df.drop(columns=[
    'currency_of_commitment',
    'due_3rd_party',
    'undisbursed_amount',
    'sold_3rd_party',
    'repaid_3rd_party',
    'end_of_period',
    'due_to_ibrd',
    'exchange_adjustment',
    'borrower\'s_obligation',
    'loans_held',
    'guarantor_country_code',
    'guarantor',
    'project_name',
    'project_id',
    'loan_number',
    'country_economy',
    'borrower',
    'last_disbursement_date',
])

# Extraindo informações das datas
def extract_date_features(df: pd.DataFrame, date_col, drop=False):
    df[date_col] = pd.to_datetime(df[date_col], errors='coerce')
    # df[f'{date_col}_year'] = df[date_col].dt.year
    # df[f'{date_col}_month'] = df[date_col].dt.month
    # df[f'{date_col}_day'] = df[date_col].dt.day
    df[f'{date_col}_timestamp'] = df[date_col].astype(np.int64, errors='ignore') // 10**9
    if drop:
        df.drop(columns=[date_col], inplace=True)
    return df



# Diferença entre as datas de pagamento
df_filtered['payment_difference'] = pd.to_datetime(df_filtered['last_repayment_date']) - pd.to_datetime(df_filtered['first_repayment_date'])
df_filtered['payment_difference'] = df_filtered['payment_difference'].dt.days
df_filtered['payment_difference'] = df_filtered['payment_difference'].fillna(0)

date_columns = [col for col in df_filtered.columns if "date" in col.lower()]
print(date_columns)
for date_col in date_columns:
    df_filtered = extract_date_features(df_filtered, date_col, drop=True)

# Convertendo colunas categóricas para minúsculas
df_filtered['region'] = df_filtered['region'].str.lower()

# Preenchendo interest rate com a mediana
df_filtered['interest_rate'] = df_filtered['interest_rate'].fillna(df_filtered['interest_rate'].median())

# O código da Namíbia é NA
df_filtered['country_code'] = df_filtered['country_code'].replace('NA', 'NAM')

# O código da Namibia é NA, mas o python interpretou como Null
df.loc[df['country_economy'] == 'Namibia', 'country_code'] = 'NAM'

# Preenchendo data faltantes com a moda
new_date_columns = [col for col in df_filtered.columns if "date" in col.lower()]
for date_col in new_date_columns:
    if df_filtered[date_col].isnull().any():
        df[date_col] = df_filtered[date_col].fillna(df_filtered[date_col].mode())

set_pct = lambda x, feature: x[feature] / x['original_principal_amount'] if x['original_principal_amount'] != 0 else 0
df_filtered['cancelled_percentage'] = df_filtered.apply(lambda x: set_pct(x, 'cancelled_amount'), axis=1)
df_filtered['repaid_percentage'] = df_filtered.apply(lambda x: set_pct(x, 'repaid_to_ibrd'), axis=1)
df_filtered['disbursed_percentage'] = df_filtered.apply(lambda x: set_pct(x, 'disbursed_amount'), axis=1)

df_filtered = pd.get_dummies(df_filtered, columns=['loan_type'], prefix_sep='_', drop_first=False, dtype=int)

['first_repayment_date', 'last_repayment_date', 'agreement_signing_date', 'board_approval_date', 'effective_date', 'closed_date']


In [13]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import LabelEncoder


numeric_features = {'interest_rate', 'original_principal_amount', 'cancelled_amount', 'disbursed_amount',
                    'repaid_to_ibrd', 'payment_difference', 'first_repayment_date_timestamp', 'last_repayment_date_timestamp',
                    'agreement_signing_date_timestamp', 'board_approval_date_timestamp',
                    'effective_date_timestamp', 'closed_date_timestamp',
                    'cancelled_percentage', 'repaid_percentage', 'disbursed_percentage'}

cat_features = {col for col in df_filtered.columns.to_list() if col not in numeric_features}


def cramers_v(confusion_matrix):
    """
    Calcula o coeficiente Cramer's V para tabelas de contingência.
    """
    chi2 = stats.chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    r, k = confusion_matrix.shape
    # Para evitar divisão por zero se min(k, r) - 1 for zero
    if min(k, r) - 1 == 0:
        return 0.0
    v = np.sqrt(chi2 / (n * (min(k, r) - 1)))
    return v

def corr_numeric_cat(df: pd.DataFrame, num_col: str, cat_col: str):
    temp_df = df[[num_col, cat_col]].dropna()
    temp_df[cat_col] = LabelEncoder().fit_transform(temp_df[cat_col])
    corr, p_val = stats.pointbiserialr(temp_df[cat_col], temp_df[num_col])
    return corr, p_val

def corr_numeric(df: pd.DataFrame, num_col1: str, num_col2: str):
    return stats.spearmanr(df[num_col1], df[num_col2])

def corr_cat(df: pd.DataFrame, cat_col1: str, cat_col2: str):
    # Remover NaNs para a tabela de contingência
    temp_df = df[[cat_col1, cat_col2]].dropna()
    contingency_table = pd.crosstab(temp_df[cat_col1], temp_df[cat_col2])
    try:
        chi2, p_val_chi2, _, _ = stats.chi2_contingency(contingency_table)
        correlation_value = cramers_v(contingency_table)
        p_value = p_val_chi2
    except ValueError as e:
        # Pode ocorrer ValueError se uma coluna da tabela de contingência for toda zero,
        # ou se houver muito poucos dados para o Qui-Quadrado
        correlation_value = np.nan
        p_value = np.nan
        method_name = f"Qui-Quadrado (Erro: {e})"
    return correlation_value, p_value

def compute_corr(df, feat1, feat2):
    if feat1 in cat_features:
        if feat2 in numeric_features:
            method = 'Point Bisserial'  
            corr_value, p_value = corr_numeric_cat(df_filtered, feat2, feat1)
        else:    
            method = 'Chi Quadrado (Cramer V)' 
            corr_value, p_value = corr_cat(df_filtered, feat1, feat2)
    else:
        if feat2 in numeric_features:
            method = "Spearman"
            corr_value, p_value = corr_numeric(df_filtered, feat2, feat1)
        else:
            method = 'Point Bisserial'
            corr_value, p_value = corr_numeric_cat(df_filtered, feat1, feat2)
    return corr_value, p_value, method

def compute_corr_with_target(df: pd.DataFrame, target: str, numerical_features: list | set, cat_features: list | set):
    features = df.columns.to_list()
    features.remove(target)
    is_target_cat = target in cat_features
    results = []

    for feature in features:
        corr_value, p_value, method_name = compute_corr(df, feature, target)
        results.append({
                'Feature': feature,
                'Valor de Associacao': corr_value,
                'P-val': p_value,
                'Method': method_name
            })
    
    return pd.DataFrame(data=results).set_index('Feature')
        

correlations = compute_corr_with_target(df_filtered, 'last_loan_status', numeric_features, cat_features) 
correlations = correlations.sort_values(by='Valor de Associacao', ascending=False).where(correlations['Valor de Associacao'].abs() > 0.2).dropna()
correlations

,Valor de Associacao,P-val,Method
Feature,,,
loan_status,0.826577,0.000000e+00,Chi Quadrado (Cramer V)
repaid_percentage,0.770669,0.000000e+00,Point Bisserial
disbursed_percentage,0.760068,0.000000e+00,Point Bisserial
loan_type_FSL,0.706360,0.000000e+00,Chi Quadrado (Cramer V)
agreement_signing_date_timestamp,0.610154,0.000000e+00,Point Bisserial
effective_date_timestamp,0.568354,0.000000e+00,Point Bisserial
interest_rate,0.520118,0.000000e+00,Point Bisserial
country_code,0.421863,1.091579e-174,Chi Quadrado (Cramer V)
loan_type_NON POOL,0.302438,3.603554e-140,Chi Quadrado (Cramer V)


In [14]:
from itertools import combinations

results = []
for feat1, feat2 in combinations(correlations.index, r=2):
    corr_value, p_value, method = compute_corr(df_filtered, feat1, feat2)
    if corr_value > 0.75:
        results.append((feat1, 
                        feat2, 
                        round(float(corr_value), 2), 
                        method))

pd.DataFrame(data=results, columns=['Feat1', 'Feat2', 'correlation', 'method'])

,Feat1,Feat2,correlation,method
0,loan_status,loan_type_FSL,0.87,Chi Quadrado (Cramer V)
1,repaid_percentage,disbursed_percentage,0.82,Spearman
2,repaid_percentage,repaid_to_ibrd,0.82,Spearman
3,loan_type_FSL,board_approval_date_timestamp,0.78,Point Bisserial
4,agreement_signing_date_timestamp,effective_date_timestamp,0.90,Spearman
5,last_repayment_date_timestamp,closed_date_timestamp,0.89,Spearman
6,last_repayment_date_timestamp,board_approval_date_timestamp,0.89,Spearman
7,closed_date_timestamp,board_approval_date_timestamp,0.99,Spearman


Features que acho que podem estar no modelo: todas que aparecem na tabela de correlação com alvo com exceção de

- disbursed_percentage: alta correlação com repaid_percentage e tem menor correlação com alvo
- repaid_to_ibrd: é a mesma feature que repaid percentage, mas transformada
- effective_date_timestamp: correlação alta com agreement_signing_date_timestamp, e esse tem maior correlação com o alvo.
- last_repayment_date_timestamp: correlação alta com closed_date_timestamp, e esse tem maior correlação com o alvo. Contudo, aquela parece ser mais "interpretável".
- board_approval_date_timestamp: alta correlação com três varipaveis correlacionadas com o alvo.
- country_code: difícil de fazer o encoding, mas talvez pudesse apelar para o ordinal e ver no que dá.


| Par de Variáveis                                                     | Correlação |
| -------------------------------------------------------------------- | :--------: |
| agreement\_signing\_date\_timestamp & effective\_date\_timestamp     |    0.919   |
| disbursed\_percentage & repaid\_percentage                           |    0.840   |
| first\_repayment\_date\_timestamp & last\_repayment\_date\_timestamp |    0.829   |
| board\_approval\_date\_timestamp & closed\_date\_timestamp           |    0.809   |
| board\_approval\_date\_timestamp & loan\_type\_FSL                   |    0.782   |
| last\_loan\_status & repaid\_percentage                              |    0.771   |
| disbursed\_percentage & last\_loan\_status                           |    0.760   |
| board\_approval\_date\_timestamp & interest\_rate                    |   –0.741   |
| interest\_rate & loan\_type\_FSL                                     |   –0.727   |
| board\_approval\_date\_timestamp & repaid\_percentage                |   –0.721   |
| loan\_type\_FSL & repaid\_percentage |   –0,715   |
| last\_loan\_status & loan\_type\_FSL |   –0,707   |


In [15]:
selected_columns  =  correlations.sort_values(by='Valor de Associacao', ascending=False).where(correlations['Valor de Associacao'].abs() > 0.2).dropna().index.to_list()
df_selected = df_filtered[selected_columns] 
df_selected = df_selected.drop(columns=['disbursed_percentage',
#                                        'agreement_signing_date_timestamp',
                                        'effective_date_timestamp',
                                        'board_approval_date_timestamp',
                                        'repaid_to_ibrd',
                                        'closed_date_timestamp',
                                        'last_repayment_date_timestamp',
 #                                       'first_repayment_date_timestamp',
                                        'country_code'], axis=1)
df_selected

,loan_status,repaid_percentage,loan_type_FSL,agreement_signing_date_timestamp,interest_rate,loan_type_NON POOL,loan_type_POOL LOAN,loan_type_SCP USD,loan_type_SNGL CRNCY,first_repayment_date_timestamp,original_principal_amount
0,Approved,0.000000,0,-9223372037,3.80,0,0,0,0,-9223372037,480000000.0
1,Approved,0.000000,0,-9223372037,3.80,0,0,0,0,-9223372037,50000000.0
2,Approved,0.000000,0,-9223372037,3.80,0,0,0,0,-9223372037,654900000.0
3,Cancelled,0.000000,0,663552000,7.10,0,1,0,0,815184000,13000000.0
4,Disbursing,0.000000,1,1175040000,0.00,0,0,0,0,418694400,32800000.0
...,...,...,...,...,...,...,...,...,...,...,...
6940,Repaid,1.000000,0,747446400,4.59,0,1,0,0,879552000,300000000.0
6941,Repaid,0.943284,0,774662400,4.85,0,1,0,0,947894400,380000000.0
6942,Repaid,0.657752,0,815097600,1.00,0,0,0,1,979516800,35000000.0
6943,Disbursed,0.041680,0,1117756800,0.96,0,0,0,1,860112000,125000000.0


### matriz de correlação pras variáveis numéricas no duckdb

```
D with base as (
  select * exclude("loan_status", "loan_type", "country_code") from '~/Downloads/2025-06-15T19_33_42+00_00_fomm.csv' f )
  select * from correlation_matrix(base);
┌───────────────────────────┬──────────────────────┬───────────────────┬──────────────────────┬──────────────────────────┬───────────────────────────┐
│             k             │ disbursed_percentage │ repaid_percentage │ cancelled_percentage │ board_approval_date_year │ first_repayment_date_year │
│          varchar          │        double        │      double       │        double        │          double          │          double           │
├───────────────────────────┼──────────────────────┼───────────────────┼──────────────────────┼──────────────────────────┼───────────────────────────┤
│ cancelled_percentage      │               -0.674 │            -0.562 │                  1.0 │                     0.08 │                     0.045 │
│ disbursed_percentage      │                  1.0 │              0.85 │               -0.674 │                    -0.36 │                    -0.312 │
│ first_repayment_date_year │               -0.312 │            -0.393 │                0.045 │                    0.927 │                       1.0 │
│ board_approval_date_year  │                -0.36 │            -0.426 │                 0.08 │                      1.0 │                     0.927 │
│ repaid_percentage         │                 0.85 │               1.0 │               -0.562 │                   -0.426 │                    -0.393 │
└───────────────────────────┴──────────────────────┴───────────────────┴──────────────────────┴──────────────────────────┴───────────────────────────┘
```

loan_type possui correlação alta com board_approval e first_repayment (0.77 e 0.72). Possivelmente eliminar essas duas últimas

loan_status nem country_code não tem correlação alta com nenhuma

disbursed_percentage e repaid_percentage tem alta correlação entre si (85%), eliminar uma das duas

first_repayment_date_year e board_approval_date_year tem alta correlação entre si (92,7%), eliminar uma das duas


# Transformação de Dados

In [16]:
# import pandas as pd
# from sklearn.preprocessing import StandardScaler

# df = df_filtered
# terminals = ['Fully Cancelled','Fully Disbursed','Fully Repaid','Fully Transferred','Terminated']
# status = pd.get_dummies(df['last_loan_status']).reindex(columns=terminals, fill_value=0)
# X = df.drop('last_loan_status', axis=1)
# cat_cols = X.select_dtypes(['object','category']).columns
# X = pd.get_dummies(X, columns=cat_cols)
# num_cols = X.select_dtypes('number').columns
# X[num_cols] = StandardScaler().fit_transform(X[num_cols])
# corr = pd.concat([status, X], axis=1).corr().loc[terminals, X.columns]
# flat = corr.unstack().reset_index()
# flat.columns = ['status','feature','correlation']
# flat_sorted = flat.sort_values('correlation', ascending=False)
# flat_sorted


In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df = df_filtered
terminals = ['Fully Cancelled','Fully Disbursed','Fully Repaid','Fully Transferred','Terminated']
status = pd.get_dummies(df['last_loan_status']).reindex(columns=terminals, fill_value=0)
X = df.drop('last_loan_status', axis=1)
cat_cols = X.select_dtypes(['object','category']).columns
X = pd.get_dummies(X, columns=cat_cols)
num_cols = X.select_dtypes('number').columns
X[num_cols] = StandardScaler().fit_transform(X[num_cols])
corr = pd.concat([status, X], axis=1).corr().loc[terminals, X.columns]
flat = corr.unstack().reset_index()
flat.columns = ['status','feature','correlation']
flat_sorted = flat.sort_values('correlation', ascending=False)
# Fixing the issue by using .str.startswith instead of .startswith
# countries = flat_sorted[flat_sorted['status'].str.startswith('country_code')]
# countries
flat_sorted

,status,feature,correlation
0,interest_rate,Fully Cancelled,NaN
1,interest_rate,Fully Disbursed,NaN
2,interest_rate,Fully Repaid,NaN
3,interest_rate,Fully Transferred,NaN
4,interest_rate,Terminated,NaN
...,...,...,...
925,loan_status_Terminated,Fully Cancelled,NaN
926,loan_status_Terminated,Fully Disbursed,NaN
927,loan_status_Terminated,Fully Repaid,NaN
928,loan_status_Terminated,Fully Transferred,NaN


In [18]:
df_11 = _dntk.execute_sql(
  'SELECT distinct loan_status from df_selected;',
  'SQL_DEEPNOTE_DATAFRAME_SQL',
  audit_sql_comment='',
  sql_cache_mode='cache_disabled',
  return_variable_type='dataframe'
)
df_11

,loan_status
0,Approved
1,Cancelled
2,Disbursing
3,Signed
4,Terminated
5,Effective
6,Disbursed
7,Repaid
8,Draft


In [19]:
status_encoding = {
    'Draft': 0,
    'Approved': 1,
    'Signed': 2,
    'Effective': 4,
    'Disbursing': 5,
    'Disbursed': 6,
    'Cancelled': 7,
    'Repaid': 8,
    'Terminated': 9,
}

df_selected['loan_status'] = df_selected['loan_status'].map(lambda x: status_encoding[x])
# df_selected = pd.get_dummies(df_selected, columns=['loan_type'], dtype='int')

In [20]:
from sklearn.preprocessing import StandardScaler

colunas_numericas = df.select_dtypes(include='number').columns
scaler = StandardScaler()
cols = df_selected.columns
scale_cols = ['repaid_percentage', 
                'interest_rate', 
                'original_principal_amount',
                'agreement_signing_date_timestamp',
                'first_repayment_date_timestamp',]
df_selected[scale_cols] = scaler.fit_transform(df_selected[scale_cols])
print(scale_cols)

['repaid_percentage', 'interest_rate', 'original_principal_amount', 'agreement_signing_date_timestamp', 'first_repayment_date_timestamp']


In [21]:
le = LabelEncoder()
y = le.fit_transform(df_filtered['last_loan_status'])
df_final = pd.concat([df_selected, pd.DataFrame(data=y, columns=['last_loan_status'])], axis=1)
df_final
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

,loan_status,repaid_percentage,loan_type_FSL,agreement_signing_date_timestamp,interest_rate,loan_type_NON POOL,loan_type_POOL LOAN,loan_type_SCP USD,loan_type_SNGL CRNCY,first_repayment_date_timestamp,original_principal_amount,last_loan_status
0,1,-1.032371,0,-1.857511,0.058461,0,0,0,0,-8.181888,1.966821,0
1,1,-1.032371,0,-1.857511,0.058461,0,0,0,0,-8.181888,-0.325679,0
2,1,-1.032371,0,-1.857511,0.058461,0,0,0,0,-8.181888,2.899283,0
3,7,-1.032371,0,0.543545,1.011680,0,1,0,0,0.148487,-0.522940,0
4,5,-1.032371,1,0.667761,-1.039185,0,0,0,0,-0.180535,-0.417379,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6940,8,1.221769,0,0.563919,0.286656,0,1,0,0,0.201902,1.007170,1
6941,8,1.093924,0,0.570528,0.361758,0,1,0,0,0.258615,1.433682,1
6942,8,0.450295,0,0.580348,-0.750331,0,0,0,1,0.284857,-0.405650,1
6943,6,-0.938418,0,0.653849,-0.761885,0,0,0,1,0.185770,0.074176,1


Código utilizado para gerar os splits 

```python
from sklearn.model_selection import RepeatedStratifiedKFold
import pickle
# Definindo repetição + divisão
n_splits = 5  # número de folds por subdivisão
n_repeats = 6  # 6 subdivisões diferentes → 6*5 = 30 folds
random_state = 42
X = df_final.drop(columns=['last_loan_status'])
y = df_final['last_loan_status']
# Lista com os 30 pares (train_idx, test_idx)
rskf = RepeatedStratifiedKFold(n_splits=n_splits, n_repeats=n_repeats, random_state=random_state)
splits = list(rskf.split(X, y))  # isso gera os 30 folds diferentes e reprodutíveis




with open("splits.pkl", "wb") as file:
    pickle.dump(splits, file)
```

Exemplo de como usar o split 

```python

with open("splits.pkl", "rb") as f:
    splits = pickle.load(f)

for i, (train_dx, test_idx) in enumerate(splits):
    X_train = df_final.iloc[train_dx]
    y_train = y[train_dx]
    X_test = df_final.iloc[test_idx]
    y_test = y[test_idx]

    # Seu código implementando o algoritmo
```



In [22]:
df_final

,loan_status,repaid_percentage,loan_type_FSL,agreement_signing_date_timestamp,interest_rate,loan_type_NON POOL,loan_type_POOL LOAN,loan_type_SCP USD,loan_type_SNGL CRNCY,first_repayment_date_timestamp,original_principal_amount,last_loan_status
0,1,-1.032371,0,-1.857511,0.058461,0,0,0,0,-8.181888,1.966821,0
1,1,-1.032371,0,-1.857511,0.058461,0,0,0,0,-8.181888,-0.325679,0
2,1,-1.032371,0,-1.857511,0.058461,0,0,0,0,-8.181888,2.899283,0
3,7,-1.032371,0,0.543545,1.011680,0,1,0,0,0.148487,-0.522940,0
4,5,-1.032371,1,0.667761,-1.039185,0,0,0,0,-0.180535,-0.417379,0
...,...,...,...,...,...,...,...,...,...,...,...,...
6940,8,1.221769,0,0.563919,0.286656,0,1,0,0,0.201902,1.007170,1
6941,8,1.093924,0,0.570528,0.361758,0,1,0,0,0.258615,1.433682,1
6942,8,0.450295,0,0.580348,-0.750331,0,0,0,1,0.284857,-0.405650,1
6943,6,-0.938418,0,0.653849,-0.761885,0,0,0,1,0.185770,0.074176,1


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=5c43b607-37dc-429f-9293-795e7b516a17' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>